In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from hashlib import sha256
from random import choice
import requests
import time
import re

In [2]:
service = Service('chromedriver.exe')
eset_registration = 'https://login.eset.com/Register'

browser_eset = webdriver.Chrome(service=service)
browser_eset.get(eset_registration)

wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="cookiebar"]/div[1]')))
button_eset_cockies = browser_eset.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[2]/div[2]/button')
button_eset_cockies.click()


In [3]:

def get_domains():
    response = requests.get("https://api.mail.tm/domains")
    return response.json()["hydra:member"][0]["domain"]

def create_account(email, password):
    response = requests.post("https://api.mail.tm/accounts", json={"address": email, "password": password})
    return response.json()

def get_token(email, password):
    response = requests.post("https://api.mail.tm/token", json={"address": email, "password": password})
    return response.json().get("token")

def get_messages(token):
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get("https://api.mail.tm/messages", headers=headers)
    return response.json()["hydra:member"]

def get_message_content(token, message_id):
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(f"https://api.mail.tm/messages/{message_id}", headers=headers)
    return response.json().get("text", "")

def extract_confirmation_link(text):
    match = re.search(r'https?://\S+', text)
    return match.group(0) if match else "Ссылка не найдена"

In [4]:
chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890!@#$%^&*()'
before = ''.join(choice(chars) for _ in range(100))

domain = get_domains()
email = f"test_{int(time.time())}@{domain}"
password = sha256(before.encode()).hexdigest()+'A'


print(f"Создание временной почты: {email}")
create_account(email, password)
token = get_token(email, password)

if token:
    print("Успешно получили токен аутентификации!")
    print("Ожидание писем...")


    email_input = browser_eset.find_element(By.XPATH, '//*[@id="email"]')
    email_input.send_keys(email)
    email_input.send_keys(Keys.ENTER)
    
    wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="password"]')))
    password_input = browser_eset.find_element(By.XPATH, '//*[@id="password"]')
    password_input.send_keys(password)
    password_input.send_keys(Keys.ENTER)
    
    time.sleep(10)  # Подождем немного, чтобы появилось письмо
    
    messages = get_messages(token)
    
    if messages:
        print("Полученные сообщения:")
        for msg in messages:
            for msg in messages:
                print(f"От: {msg['from']['address']}, Тема: {msg['subject']}")
                message_text = get_message_content(token, msg['id'])
                confirmation_link = extract_confirmation_link(message_text).replace(']','')
                print(f"Ссылка для подтверждения: {confirmation_link}")
                browser_eset.get(confirmation_link)
    else:
        print("Сообщений пока нет.")
else:
    print("Ошибка получения токена.")


Создание временной почты: test_1742570555@indigobook.com
Успешно получили токен аутентификации!
Ожидание писем...
Полученные сообщения:
От: info@product.eset.com, Тема: Подтверждение учетной записи
Ссылка для подтверждения: https://login.eset.com/link/confirmregistration?token=7bb5acce-7f5b-4380-81f4-51379233098b


In [5]:
wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div/div/div[2]/div/div/div/button')))
browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div/div/div[2]/div/div/div/button').click()

wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="license-add-new-slides"]/div/ion-slide[1]/div/div/div[3]/div/button')))
browser_eset.find_element(By.XPATH, '//*[@id="license-add-new-slides"]/div/ion-slide[1]/div/div/div[3]/div/button').click()

browser_eset.find_element(By.XPATH, '//*[@id="license-add-new-slides"]/div/ion-slide[1]/div/button').click()

wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div/div/main/div/div/div[2]/div/div/div/div/div/div/div/div[1]/div/div/div/button')))
browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div/div/main/div/div/div[2]/div/div/div/div/div/div/div/div[1]/div/div/div/button').click()
browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div/div/main/div/div/div[2]/div/div/div/div/button[1]').click()

wait(browser_eset, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/div/div/button')))
browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/div/div/button').click()
browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div[2]/div/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-content"]/div/div[1]/div/main/div/div/div[2]/div/div/div/div/div[2]/div/div/div/button"}
  (Session info: chrome=134.0.6998.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF78BFDFE45+26629]
	(No symbol) [0x00007FF78BF46010]
	(No symbol) [0x00007FF78BDD931A]
	(No symbol) [0x00007FF78BE2F8E7]
	(No symbol) [0x00007FF78BE2FB1C]
	(No symbol) [0x00007FF78BE834A7]
	(No symbol) [0x00007FF78BE57AEF]
	(No symbol) [0x00007FF78BE80169]
	(No symbol) [0x00007FF78BE57883]
	(No symbol) [0x00007FF78BE20550]
	(No symbol) [0x00007FF78BE21803]
	GetHandleVerifier [0x00007FF78C3372BD+3529853]
	GetHandleVerifier [0x00007FF78C34DA22+3621858]
	GetHandleVerifier [0x00007FF78C3424D3+3575443]
	GetHandleVerifier [0x00007FF78C0AB77A+860474]
	(No symbol) [0x0

In [ ]:
key = browser_eset.find_element(By.XPATH, '//*[@id="main-content"]/div/div/div/main/div/div/div[2]/div/div/div[3]/div[1]/div/div/div/div[1]/div[2]/div[6]/div/div').find_element(By.TAG_NAME,'p').text
key
